# Navier-Stokes simulation in 2D
Currently only the simulation of a pressure driven lid is possible. This can be extended by adding more possibilities to the function 'set_BC'.

In [ ]:
#%matplotlib notebook
import time
import numpy as np
import scipy as sci

from types import SimpleNamespace

from functions import *
from constants import *
from simulation import *

start_time = time.time()

# Initial conditions 
const = SimpleNamespace()

const.Re = 1e2      # Reynolds number
const.dt = 1e-2     # time step
const.tf = 4e0      # final time
const.nx = 50       # number of x-gridpoints
const.ny = 50       # number of y-gridpoints
const.nsteps = 10   # step interval for graphic output
const.lx = 1        # width of box
const.ly = 1        # height of box
    
const.rho_arrow = 20 # Density of arrows in quiver plot per unit length l

## Set other constants (incl. grid) which are determined by initial conditions specified above
const = constants(const)

# IC's
data = SimpleNamespace()
data = initialisation(const, data)

# BC's
## North, East, South, West
bc = SimpleNamespace()
bc = set_BC(const, bc, 'lid')

# Boundary matrices
const.Ubc , const.Vbc = set_BM(const, bc) # Maybe change const. to bc.

# Operator matrices on P, U, V, Q respectively
LP = SimpleNamespace()
LP = laplacian(const, LP)
LP = cholesky_decomposition(LP)

# Simulation
simulation_data = simulation(const, bc, LP, data)

end_time = time.time()
print('Runtime', np.round(end_time - start_time,2), 's')

# Preliminary object function

In [ ]:
np.shape(data.V)

In [ ]:
# Circular object
scale = 1/5
R = const.lx*scale/2
cx, cy = const.lx/2, const.ly/2

indices_Pgrid = ((const.X_ave-cx)**2 /3 + (const.Y_ave-cy)**2 <= R**2) # devided by three to see clear differences later on # The object is projected on the pressure grid (grid points in middle of cell)
indices_grid = ((const.X-cx)**2 + (const.Y-cy)**2 <= R**2)


In [ ]:
indices_Ugrid = np.zeros((np.shape(data.U)), dtype=bool)
indices_Vgrid = np.zeros((np.shape(data.V)), dtype=bool)

# Firstly shape[1] as this is the x-coordinate and shape[0] secondly as this is the y-coordinate
# range starts at 1 in order to prevent looking to non-existent elements. (Here it is okay to start from 1 for both x and y as the object is not at the boundary of the domain)
for x in range(1, np.shape(indices_Pgrid)[1]):
    for y in range(1, np.shape(indices_Pgrid)[0]):
        if indices_Pgrid[y,x]:
            if indices_Pgrid[y, x-1]:
                indices_Ugrid[y, x] = True
            if indices_Pgrid[y+1, x]:
                indices_Vgrid[y, x] = True # Indices kloppen hier nog niet
visualise_matrix(indices_Ugrid, 'Ugrid')
visualise_matrix(indices_Vgrid, 'Vgrid')

In [ ]:
# Visual check of object
'''
indices_Pgrid = indices_Pgrid*1
indices_Pgrid[6,8] = 10
indices_Pgrid[6,7] = 20

indices_Pgrid[10,20] = 10
indices_Pgrid[9,20] = 20
'''
visualise_matrix(indices_Pgrid,'indices_Pgrid')
visualise_matrix(indices_grid,'indices_grid')

# Visual representation of grid
visualise_matrix((const.X_ave-cx)**2 + (const.Y_ave-cy)**2,'Translated Pgrid')
visualise_matrix((const.X_ave)**2 + (const.Y_ave)**2,'Original Pgrid')

visualise_matrix((const.X-cx)**2 + (const.Y-cy)**2,'Translated grid')
visualise_matrix((const.X)**2 + (const.Y)**2,'Original grid')